In [2]:
import pandas as pd
import numpy as np
import os
import json
from datasets import load_dataset
import datasets

In [ ]:
models=['gemini','Qwen2.5-Omni-7B','Kimi-Audio-7B-Instruct','Qwen2-Audio-7B-Instruct']

In [ ]:
ds = load_dataset("Yi3852/MuCUE", revision="main", trust_remote_code=True).cast_column("audio", datasets.Audio(decode=False))
data_names=list(ds.keys())

In [ ]:
accuracy_dict = {}
for model_name in models:
    for data_name in data_names:
        save_path = '../outputs/{}_{}.json'.format(model_name, data_name)
        with open(save_path, 'r') as file:
            data = json.load(file)
        
        question_num = 0
        correct_num = 0
        for item in data:
            if "pred_choice" in item: 
                question_num += 1
                if item["pred_choice"] == item["correct_choice"]:
                    correct_num += 1
        
        assert question_num >= 200, f"Error: {model_name} on {data_name} has fewer than 300 questions."
        
        accuracy = 100 * (correct_num / question_num)
        
        if data_name not in accuracy_dict:
            accuracy_dict[data_name] = {}
        accuracy_dict[data_name][model_name] = accuracy

accuracy_df = pd.DataFrame(accuracy_dict).T  
print(accuracy_df)
accuracy_df.to_csv("accuracy_results.csv")